In [1]:
import glob
import numpy as np
import os
import PIL
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from datetime import datetime
from keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Dropout, Activation, Flatten, Dense, UpSampling2D
from keras.models import *
from keras.optimizers import SGD,Adam,Adagrad,Nadam,RMSprop
from keras.callbacks import TensorBoard,ModelCheckpoint
from keras.models import Model

np.random.seed(42)

/Users/Tommy/opt/anaconda3/envs/python36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/Tommy/opt/anaconda3/envs/python36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/Tommy/opt/anaconda3/envs/python36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/Tommy/opt/anaconda3/e

In [6]:
#load data
_asset_path = '../datasets and figures/asset2/dog_cat/'
train_dir = _asset_path+'training_data/'
test_dir = _asset_path+'test_data/'

from sklearn.preprocessing import LabelEncoder
import pandas as pd
IMG_DIM = (32, 32)


train_files = glob.glob(train_dir+"*") 
train_imgs = [img_to_array(load_img(img, target_size=IMG_DIM)) for img in train_files]
train_imgs = np.array(train_imgs)

test_files = glob.glob(test_dir+'*')
test_imgs = [img_to_array(load_img(img, target_size=IMG_DIM)) for img in test_files]
test_imgs = np.array(test_imgs)
print(train_imgs.shape)


(3000, 32, 32, 3)


In [9]:
# def simpleCNN(input_shape=(48, 48, 3), classes=1):
input_shape=(32, 32, 3)
img_input = Input(shape=input_shape)
# bn_axis = 3

x = Conv2D(64, (3, 3), activation='relu', padding='same')(img_input)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(16, (3, 3), activation='relu', padding='same')(encoded)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

model = Model(img_input, decoded)
model.summary()
#     return model

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 32, 32, 64)        1792      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 16, 16, 32)        18464     
_________________________________________________________________
batch_normalization_6 (Batch (None, 16, 16, 32)        128       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 8, 8, 32)          0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 8, 8, 16)          4624

In [10]:
model.compile(optimizer='adam', loss='binary_crossentropy')

In [11]:
hist = model.fit(train_imgs, train_imgs,
                epochs=5,
                batch_size=256,
                shuffle=True,
                validation_split=0.1)

Instructions for updating:
Use tf.cast instead.
Train on 2700 samples, validate on 300 samples
Epoch 1/5
2700/2700 [==============================] - 17s 6ms/step - loss: -217.6224 - val_loss: -1631.9128
Epoch 2/5
2700/2700 [==============================] - 16s 6ms/step - loss: -593.9862 - val_loss: -1645.4814
Epoch 3/5
2700/2700 [==============================] - 16s 6ms/step - loss: -911.9609 - val_loss: -1731.0789
Epoch 4/5
2700/2700 [==============================] - 15s 6ms/step - loss: -1209.2124 - val_loss: -1695.8119
Epoch 5/5
2700/2700 [==============================] - 15s 6ms/step - loss: -1476.5948 - val_loss: -1606.3620


## (a) funcation model on dog&cat dataset

In [20]:
from keras.layers import Input, Dense
from keras.models import Model
from sklearn.preprocessing import LabelEncoder
import pandas as pd
np.random.seed(1337)

In [21]:
_asset_path = '../datasets and figures/asset2/dog_cat/'


# _asset_path = 'asset2/dog_cat/'
train_dir = _asset_path+'training_data/'
IMG_DIM = (150, 150)


train_files = glob.glob(train_dir+"*") 
train_imgs = [img_to_array(load_img(img, target_size=IMG_DIM)) for img in train_files]
train_imgs = np.array(train_imgs)
train_imgs = train_imgs.reshape((len(train_imgs), np.prod(train_imgs.shape[1:])))

test_files = glob.glob(test_dir+'*')
test_imgs = [img_to_array(load_img(img, target_size=IMG_DIM)) for img in test_files]
test_imgs = np.array(test_imgs)
test_imgs = test_imgs.reshape((len(test_imgs), np.prod(test_imgs.shape[1:])))

print(train_imgs.shape)

(3000, 67500)


In [22]:
input_img = Input(shape = (67500,))
encoded = Dense(784,activation='relu')(input_img)
encoded = Dense(128,activation='relu')(encoded)
encoded = Dense(64,activation='relu')(encoded)
encoder_output  = Dense(10)(encoded)

encoded = Dense(64,activation='relu')(encoder_output)
encoded = Dense(128,activation='relu')(encoded)
encoded = Dense(784,activation='relu')(encoded)
encoded = Dense(67500,activation='tanh')(encoded)

autoencoder = Model(inputs = input_img, outputs = encoded)
autoencoder.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 67500)             0         
_________________________________________________________________
dense_9 (Dense)              (None, 784)               52920784  
_________________________________________________________________
dense_10 (Dense)             (None, 128)               100480    
_________________________________________________________________
dense_11 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_12 (Dense)             (None, 10)                650       
_________________________________________________________________
dense_13 (Dense)             (None, 64)                704       
_________________________________________________________________
dense_14 (Dense)             (None, 128)               8320

In [23]:
autoencoder.predict(test_imgs)
autoencoder.compile(optimizer='adam',loss = 'mse')
autoencoder.fit(train_imgs,train_imgs,epochs=10,batch_size = 256,shuffle=True,validation_split=0.1)

Train on 2700 samples, validate on 300 samples
Epoch 1/10
2700/2700 [==============================] - 53s 19ms/step - loss: 17358.7635 - val_loss: 17537.7530
Epoch 2/10
2700/2700 [==============================] - 43s 16ms/step - loss: 17336.7343 - val_loss: 17537.7530
Epoch 3/10
2700/2700 [==============================] - 38s 14ms/step - loss: 17336.7345 - val_loss: 17537.7530
Epoch 4/10
2700/2700 [==============================] - 40s 15ms/step - loss: 17336.7344 - val_loss: 17537.7530
Epoch 5/10
2700/2700 [==============================] - 35s 13ms/step - loss: 17336.7343 - val_loss: 17537.7530
Epoch 6/10
2700/2700 [==============================] - 36s 13ms/step - loss: 17336.7342 - val_loss: 17537.7530
Epoch 7/10
2700/2700 [==============================] - 46s 17ms/step - loss: 17336.7339 - val_loss: 17537.7530
Epoch 8/10
2700/2700 [==============================] - 57s 21ms/step - loss: 17336.7347 - val_loss: 17537.7530
Epoch 9/10
2700/2700 [==============================] - 4

## (b)Sequential model on dog&cat datasets

In [24]:
# _asset_path = 'asset2/dog_cat/'

train_dir = _asset_path+'training_data/'
IMG_DIM = (32,32)


train_files = glob.glob(train_dir+"*") 
train_imgs = [img_to_array(load_img(img, target_size=IMG_DIM)) for img in train_files]
train_imgs = np.array(train_imgs)
train_imgs = train_imgs.reshape((len(train_imgs), np.prod(train_imgs.shape[1:])))

test_files = glob.glob(test_dir+'*')
test_imgs = [img_to_array(load_img(img, target_size=IMG_DIM)) for img in test_files]
test_imgs = np.array(test_imgs)
test_imgs = test_imgs.reshape((len(test_imgs), np.prod(test_imgs.shape[1:])))
print(train_imgs.shape)

(3000, 3072)


In [25]:
model = Sequential()
# input_img = Input(shape = (67500,))
model.add(Dense(3072, activation='relu',input_shape=(3072,)))
model.add(Dense(784, activation='relu'))

model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(64, activation='relu'))

model.add(Dense(784, activation='relu'))
model.add(Dense(3072, activation='relu'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_17 (Dense)             (None, 3072)              9440256   
_________________________________________________________________
dense_18 (Dense)             (None, 784)               2409232   
_________________________________________________________________
dense_19 (Dense)             (None, 64)                50240     
_________________________________________________________________
dense_20 (Dense)             (None, 10)                650       
_________________________________________________________________
dense_21 (Dense)             (None, 64)                704       
_________________________________________________________________
dense_22 (Dense)             (None, 784)               50960     
_________________________________________________________________
dense_23 (Dense)             (None, 3072)             

In [26]:
model.compile(optimizer='adam',loss = 'mse')
model.fit(train_imgs,train_imgs,epochs=5,batch_size = 256,shuffle=True,validation_split=0.1)
model.predict(test_imgs)

Train on 2700 samples, validate on 300 samples
Epoch 1/5
2700/2700 [==============================] - 5s 2ms/step - loss: 21268.0565 - val_loss: 8475.3497
Epoch 2/5
2700/2700 [==============================] - 4s 2ms/step - loss: 7396.5446 - val_loss: 6999.5694
Epoch 3/5
2700/2700 [==============================] - 4s 1ms/step - loss: 6645.0522 - val_loss: 6544.6064
Epoch 4/5
2700/2700 [==============================] - 4s 1ms/step - loss: 6329.6957 - val_loss: 6406.4765
Epoch 5/5
2700/2700 [==============================] - 4s 1ms/step - loss: 6225.4264 - val_loss: 6317.5285


array([[ 96.10975 ,  89.71912 ,  84.958115, ..., 105.767105,  92.75763 ,
         93.832535],
       [125.826485, 117.4976  , 111.24248 , ..., 138.35501 , 121.536224,
        122.8392  ],
       [151.65239 , 142.57458 , 135.46207 , ..., 165.96257 , 146.55322 ,
        148.43866 ],
       ...,
       [122.57028 , 114.439285, 108.26966 , ..., 134.87233 , 118.27931 ,
        119.389694],
       [111.77508 , 104.86798 ,  99.59521 , ..., 122.5067  , 107.86435 ,
        109.349594],
       [174.46878 , 163.5136  , 155.11195 , ..., 191.45294 , 168.34673 ,
        170.3226  ]], dtype=float32)

# Ex1 funcation model on butterfly dataset


In [7]:
from keras.layers import Input, Dense
from keras.models import Model
from sklearn.preprocessing import LabelEncoder
import pandas as pd

import glob
import numpy as np
import os
import PIL
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from datetime import datetime
from keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Dropout, Activation, Flatten, Dense, UpSampling2D
from keras.models import *
from keras.optimizers import SGD,Adam,Adagrad,Nadam,RMSprop
from keras.callbacks import TensorBoard,ModelCheckpoint
from keras.models import Model
np.random.seed(1337)

In [8]:
IMG_DIM = (32,32)
# _data_path = './asset3/data/'
train_data_dir = _data_path+"train1/"
# valid_data_dir = _data_path+"male/"
test_data_dir = _data_path+"test/"
train_files = glob.glob(train_data_dir+"*") 
print(len(train_files))
train_imgs = [img_to_array(load_img(img, target_size=IMG_DIM)) for img in train_files]
train_imgs = np.array(train_imgs)
train_imgs = train_imgs.reshape((len(train_imgs), np.prod(train_imgs.shape[1:])))

test_files = glob.glob(test_data_dir+'*')
test_imgs = [img_to_array(load_img(img, target_size=IMG_DIM)) for img in test_files]
test_imgs = np.array(test_imgs)
test_imgs = test_imgs.reshape((len(test_imgs), np.prod(test_imgs.shape[1:])))

print(train_imgs.shape)

244
(244, 3072)


In [9]:
input_img = Input(shape = (3072,))
encoded = Dense(784,activation='relu')(input_img)
encoded = Dense(128,activation='relu')(encoded)
encoded = Dense(64,activation='relu')(encoded)
encoder_output  = Dense(10)(encoded)

encoded = Dense(64,activation='relu')(encoder_output)
encoded = Dense(128,activation='relu')(encoded)
encoded = Dense(784,activation='relu')(encoded)
encoded = Dense(3072,activation='tanh')(encoded)

autoencoder = Model(inputs = input_img, outputs = encoded)
autoencoder.summary()




_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 784)               2409232   
_________________________________________________________________
dense_2 (Dense)              (None, 128)               100480    
_________________________________________________________________
dense_3 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_4 (Dense)              (None, 10)                650       
_________________________________________________________________
dense_5 (Dense)              (None, 64)                704       
_________________________________________________________________
dense_6 (Dense)              (None, 128)               8320      
_______

In [10]:
autoencoder.compile(optimizer='adam',loss = 'binary_crossentropy')
autoencoder.fit(train_imgs,train_imgs,epochs=10,batch_size = 10,shuffle=True,validation_split=0.1)
autoencoder.predict(test_imgs)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 219 samples, validate on 25 samples
Epoch 1/10
219/219 [==============================] - 3s 13ms/step - loss: -704.9290 - val_loss: -991.2557
Epoch 2/10
219/219 [==============================] - 1s 6ms/step - loss: -1067.0822 - val_loss: -1130.1061
Epoch 3/10
219/219 [==============================] - 1s 5ms/step - loss: -1148.2071 - val_loss: -1173.4436
Epoch 4/10
219/219 [==============================] - 1s 5ms/step - loss: -1178.0336 - val_loss: -1208.4757
Epoch 5/10
219/219 [==============================] - 1s 5ms/step - loss: -1225.1790 - val_loss: -1262.9855
Epoch 6/10
219/219 [==============================] - 1s 6ms/step - loss: -1255.4629 - val_loss: -1265.8196
Epoch 7/10
219/219 [==============================] - 1s 6ms/step - loss: -1288.2391 - val_loss: -1300.7519
Epoch 8/10
219/219 [==============================] - 1s 5ms/step - loss: -1309.0384 - val_loss: -1299.

# Ex2  Sequential model on butterfly dataset

In [11]:
model = Sequential()
# input_img = Input(shape = (67500,))
model.add(Dense(3072, activation='relu',input_shape=(3072,)))
model.add(Dense(784, activation='relu'))

model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(64, activation='relu'))

model.add(Dense(784, activation='relu'))
model.add(Dense(3072, activation='relu'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 3072)              9440256   
_________________________________________________________________
dense_10 (Dense)             (None, 784)               2409232   
_________________________________________________________________
dense_11 (Dense)             (None, 64)                50240     
_________________________________________________________________
dense_12 (Dense)             (None, 10)                650       
_________________________________________________________________
dense_13 (Dense)             (None, 64)                704       
_________________________________________________________________
dense_14 (Dense)             (None, 784)               50960     
_________________________________________________________________
dense_15 (Dense)             (None, 3072)              2411520   
Total para

In [12]:
model.compile(optimizer='adam',loss = 'binary_crossentropy')
model.fit(train_imgs,train_imgs,epochs=5,batch_size = 10,shuffle=True,validation_split=0.1)
model.predict(test_imgs)

Train on 219 samples, validate on 25 samples
Epoch 1/5
219/219 [==============================] - 5s 21ms/step - loss: -699.4843 - val_loss: -801.6470
Epoch 2/5
219/219 [==============================] - 3s 13ms/step - loss: -806.0410 - val_loss: -814.3039
Epoch 3/5
219/219 [==============================] - 3s 12ms/step - loss: -815.7404 - val_loss: -817.9398
Epoch 4/5
219/219 [==============================] - 3s 13ms/step - loss: -817.3713 - val_loss: -820.6753
Epoch 5/5
219/219 [==============================] - 3s 13ms/step - loss: -824.0845 - val_loss: -837.3964
